In [1]:
#!python3 -m spacy download en_core_web_lg
# warning: takes 80 years on public wifi
# !pip install spacy

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import re
from nltk.stem.wordnet import WordNetLemmatizer

from typing import List, Set

import spacy

from nltk import FreqDist

from math import log

In [ ]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
def remove_tags(text: str) -> str:    
    return re.sub('&lt;/?.*?&gt;', '', text)

def remove_special_chars_and_digits(text: str) -> str:
    return re.sub('(\\d|\\W)+', '', text)

def remove_punctuation(text: str) -> str:
    return re.sub('[^a-zA-Z]', '', str(text))

In [ ]:
def apply_lemmatization(in_text):
    # Lemmatization
    lem = WordNetLemmatizer()
    word_list = nltk.word_tokenize(in_text)
    print(word_list)
    output = ' '.join([lem.lemmatize(w) for w in word_list])
    return output

In [ ]:
def get_lemmas(text: str, stopwords: Set[str]) -> List[str]:
    initial = [remove_tags(remove_special_chars_and_digits(remove_punctuation(x.lemma_.lower()))) for x in nlp(text)]
    return [x for x in initial if x not in stopwords]

In [ ]:
stop_words = stopwords.words('english')

def clean_please(some_data):
    """
    """
    re_punk = re.compile('[%s]' % re.escape(string.punctuation))
    lemmas = apply_lemmatization(some_data)
    tokens = word_tokenize(lemmas)
    stripped_tkns = [re_punk.sub('', wxy) for wxy in tokens]
    lower = [word.lower() for word in stripped_tkns]
    st0p = [word for word in lower if not word in stop_words]

    while "" in st0p:
        st0p.remove("")
    return st0p

In [ ]:
TDM = pd.read_csv("../week2/document_matrix.csv") #term_document_matrix

In [ ]:
reviews = pd.read_csv("/Users/kerry/Projects/msds453/random_nlp/assign0/toxicAvenger/reviews.csv", sep="|",
                     lineterminator=">", engine="c")

In [ ]:
stg = ""
for rev in reviews.review:
    stg += rev
    
stg = stg.translate(str.maketrans('', '', string.punctuation))
stg = stg.replace("\n", " ").replace("  ", " ")

In [ ]:
stg = " ".join([word.lower() for word in stg.split(" ")])

In [ ]:
words = get_lemmas(stg, stop_words)

In [ ]:
fdist1 = FreqDist(words)

In [ ]:
TDM.rename(columns={"Unnamed: 0":"terms"}, inplace=True)

In [ ]:
#df['sum']=df.sum(axis=1) # jk
TDM['n_doc_freq_of_term'] = TDM.astype(bool).sum(axis=1)

In [ ]:
TDM.set_index("terms")

In [ ]:
def TF_IDF(value, idx):
    """
    value is a series object passed with axis=1
    """
    idf = log(10/value.n_doc_freq_of_term)+1
    tf = value[f'Document {idx}']/len(fdist1)
    return idf*tf

In [ ]:
abc = {'terms':TDM.terms,
       'Document 1':TDM.apply(lambda x: TF_IDF(x, '1'), axis=1),
       'Document 2':TDM.apply(lambda x: TF_IDF(x, '2'), axis=1),
       'Document 3':TDM.apply(lambda x: TF_IDF(x, '3'), axis=1),
       'Document 4':TDM.apply(lambda x: TF_IDF(x, '4'), axis=1),
       'Document 5':TDM.apply(lambda x: TF_IDF(x, '5'), axis=1),
       'Document 6':TDM.apply(lambda x: TF_IDF(x, '6'), axis=1),
       'Document 7':TDM.apply(lambda x: TF_IDF(x, '7'), axis=1),
       'Document 8':TDM.apply(lambda x: TF_IDF(x, '8'), axis=1),
       'Document 9':TDM.apply(lambda x: TF_IDF(x, '9'), axis=1)}

In [ ]:
df = pd.DataFrame(abc)

In [ ]:
df.set_index('terms', inplace=True)

In [ ]:
df

In [ ]:
df.idxmax()

In [ ]:
terms = list(TDM['terms'])
terms.remove("film")
terms.remove("toxic")

In [ ]:
TDM.set_index("terms", inplace=True)

In [ ]:
TDM2 = TDM.drop(index=["film", "toxic"], axis=0)

In [ ]:
abc2 = {'terms':terms,
       'Document 1':TDM2.apply(lambda x: TF_IDF(x, '1'), axis=1),
       'Document 2':TDM2.apply(lambda x: TF_IDF(x, '2'), axis=1),
       'Document 3':TDM2.apply(lambda x: TF_IDF(x, '3'), axis=1),
       'Document 4':TDM2.apply(lambda x: TF_IDF(x, '4'), axis=1),
       'Document 5':TDM2.apply(lambda x: TF_IDF(x, '5'), axis=1),
       'Document 6':TDM2.apply(lambda x: TF_IDF(x, '6'), axis=1),
       'Document 7':TDM2.apply(lambda x: TF_IDF(x, '7'), axis=1),
       'Document 8':TDM2.apply(lambda x: TF_IDF(x, '8'), axis=1),
       'Document 9':TDM2.apply(lambda x: TF_IDF(x, '9'), axis=1)}

In [ ]:
df2 = pd.DataFrame(abc2)

In [ ]:
df2.set_index('terms', inplace=True)

In [ ]:
df2.idxmax()